In [1]:
import argparse
import json
import datetime as dt
import numpy as np
from scipy.io.wavfile import write

import torch

import params
from model import GradTTS
from text import text_to_sequence, cmudict
from text.symbols import symbols
from utils import intersperse

import sys
sys.path.append('hifi/')
from env import AttrDict
from models import Generator as HiFiGAN


HIFIGAN_CONFIG = './checkpts/hifigan-config.json'
HIFIGAN_CHECKPT = './checkpts/hifigan.pt'

In [2]:
speaker_id = None
checkpoint = "logs/new_exp/grad_100.pt"

In [9]:
if not isinstance(speaker_id, type(None)):
    assert params.n_spks > 1, "Ensure you set right number of speakers in `params.py`."
    spk = torch.LongTensor([speaker_id]).cuda()
else:
    spk = None

print('Initializing Grad-TTS...')
generator = GradTTS(len(symbols)+1, params.n_spks, params.spk_emb_dim,
                    params.n_enc_channels, params.filter_channels,
                    params.filter_channels_dp, params.n_heads, params.n_enc_layers,
                    params.enc_kernel, params.enc_dropout, params.window_size,
                    params.n_feats, params.dec_dim, params.beta_min, params.beta_max, params.pe_scale)
generator.load_state_dict(torch.load(checkpoint, map_location=lambda loc, storage: loc))
_ = generator.cuda().eval()
print(f'Number of parameters: {generator.nparams}')

print('Initializing HiFi-GAN...')
with open(HIFIGAN_CONFIG) as f:
    h = AttrDict(json.load(f))
vocoder = HiFiGAN(h)
vocoder.load_state_dict(torch.load(HIFIGAN_CHECKPT, map_location=lambda loc, storage: loc)['generator'])
_ = vocoder.cuda().eval()
vocoder.remove_weight_norm()

Initializing Grad-TTS...
Number of parameters: 14829464
Initializing HiFi-GAN...
Removing weight norm...


In [20]:
texts = ["По словам CEO и главного инженера SpaceX Илона Маска, Starship в будущем заменит Falcon 9 и Falcon Heavy, грузовой и пилотируемый Dragon V2[11].;"]

In [21]:
timesteps = 10

In [22]:
with torch.no_grad():
    for i, text in enumerate(texts):
        print(f'Synthesizing {i} text...', end=' ')
        x = torch.LongTensor(intersperse(text_to_sequence(text), len(symbols))).cuda()[None]
        x_lengths = torch.LongTensor([x.shape[-1]]).cuda()

        t = dt.datetime.now()
        y_enc, y_dec, attn = generator.forward(x, x_lengths, n_timesteps=timesteps, temperature=1.5,
                                               stoc=False, spk=spk, length_scale=0.91)
        t = (dt.datetime.now() - t).total_seconds()
        print(f'Grad-TTS RTF: {t * 22050 / (y_dec.shape[-1] * 256)}')

        audio = (vocoder.forward(y_dec).cpu().squeeze().clamp(-1, 1).numpy() * 32768).astype(np.int16)

        write(f'./out/sample_2.wav', 22050, audio)

Synthesizing 0 text... Grad-TTS RTF: 0.06282278315946349


In [15]:
from data import TextMelDataset, TextMelBatchCollate
from torch.utils.data import DataLoader

In [16]:
train_filelist_path = params.train_filelist_path
valid_filelist_path = params.valid_filelist_path
add_blank = params.add_blank

log_dir = params.log_dir
n_epochs = params.n_epochs
batch_size = params.batch_size
out_size = params.out_size
learning_rate = params.learning_rate
random_seed = params.seed

nsymbols = len(symbols) + 1 if add_blank else len(symbols)
n_enc_channels = params.n_enc_channels
filter_channels = params.filter_channels
filter_channels_dp = params.filter_channels_dp
n_enc_layers = params.n_enc_layers
enc_kernel = params.enc_kernel
enc_dropout = params.enc_dropout
n_heads = params.n_heads
window_size = params.window_size

n_feats = params.n_feats
n_fft = params.n_fft
sample_rate = params.sample_rate
hop_length = params.hop_length
win_length = params.win_length
f_min = params.f_min
f_max = params.f_max

dec_dim = params.dec_dim
beta_min = params.beta_min
beta_max = params.beta_max
pe_scale = params.pe_scale

In [17]:
train_dataset = TextMelDataset(
    train_filelist_path, add_blank, n_fft, n_feats, sample_rate, hop_length, win_length, f_min, f_max
)

In [18]:
batch_collate = TextMelBatchCollate()
loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                        collate_fn=batch_collate, drop_last=True,
                        num_workers=4, shuffle=False)

In [19]:
%%timeit

for batch in loader:
    pass


4.28 s ± 253 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
text = "По словам CEO и главного инженера SpaceX Илона Маска, Starship в будущем заменит Falcon 9 и Falcon Heavy, грузовой и пилотируемый Dragon V2[11].;"

In [4]:
text_to_sequence(text, ["russian_cleaners"])

По словам CEO и главного инженера SpaceX Илона Маска, Starship в будущем заменит Falcon 9 и Falcon Heavy, грузовой и пилотируемый Dragon V2[11].;
None
PA SLAVAM S0IA I GLAVNAVA INZHYN0IRA SPAS0IKS ILANA MASKA STARSHY F BUDUSH0IM ZAM0IN0IT FALKAN D0IV0IT0 I FALKAN KH0IV0I GRUZAVAJ0 I P0ILAT0IRUJ0IMYJ0 DRAGAN F TF GRUZAVAJ0 I P0ILAT0IRUJ0IMYJ0 F AD0INATSAT0


[96,
 65,
 64,
 100,
 88,
 65,
 112,
 65,
 90,
 64,
 100,
 81,
 65,
 64,
 81,
 64,
 78,
 88,
 65,
 112,
 92,
 65,
 112,
 65,
 64,
 81,
 92,
 116,
 9,
 114,
 92,
 81,
 98,
 65,
 64,
 100,
 96,
 65,
 100,
 81,
 84,
 100,
 64,
 81,
 88,
 65,
 92,
 65,
 64,
 90,
 65,
 100,
 84,
 65,
 64,
 100,
 104,
 65,
 98,
 100,
 9,
 114,
 64,
 76,
 64,
 67,
 110,
 69,
 110,
 100,
 9,
 81,
 90,
 64,
 116,
 65,
 90,
 81,
 92,
 81,
 104,
 64,
 76,
 65,
 88,
 84,
 65,
 92,
 64,
 69,
 81,
 112,
 81,
 104,
 64,
 81,
 64,
 76,
 65,
 88,
 84,
 65,
 92,
 64,
 84,
 9,
 81,
 112,
 81,
 64,
 78,
 98,
 110,
 116,
 65,
 112,
 65,
 11,
 64,
 81,
 64,
 96,
 81,
 88,
 65,
 104,
 81,
 98,
 110,
 11,
 81,
 90,
 114,
 11,
 64,
 69,
 98,
 65,
 78,
 65,
 92,
 64,
 76,
 64,
 104,
 76,
 64,
 78,
 98,
 110,
 116,
 65,
 112,
 65,
 11,
 64,
 81,
 64,
 96,
 81,
 88,
 65,
 104,
 81,
 98,
 110,
 11,
 81,
 90,
 114,
 11,
 64,
 76,
 64,
 65,
 69,
 81,
 92,
 65,
 104,
 100,
 65,
 104,
 1]

In [4]:
with open(r"dataset/transcript.txt", "r", encoding="utf-8") as t:
    texts = t.read()

In [ ]:
texts

In [12]:
a2 = [("dataset/" + i.split("|")[1], i.split("|")[1]) for i in texts.split("\n") if len(i.split("|")[:2]) == 2]

In [13]:
test_p = 0.1
val_p = 0.05

max_c = 200


In [14]:
a2

[('dataset/За столицей мудрого царя Соломона шелестел по склонам холмов густой лес. С его опушки запутанные тропинки вели на поляну,',
  'За столицей мудрого царя Соломона шелестел по склонам холмов густой лес. С его опушки запутанные тропинки вели на поляну,'),
 ('dataset/где происходили свидания Ариэля и Тамары. Ему было около четырнадцати лет, и ей тоже.',
  'где происходили свидания Ариэля и Тамары. Ему было около четырнадцати лет, и ей тоже.'),
 ('dataset/Но Ариэль был сыном знатного иерусалимца, одного из любимейших советников премудрого царя,',
  'Но Ариэль был сыном знатного иерусалимца, одного из любимейших советников премудрого царя,'),
 ('dataset/и его волосы были черны, как ночь, а глаза — как уголь. А Тамара жила за городом, потому что ее отцу, иноплеменнику,',
  'и его волосы были черны, как ночь, а глаза — как уголь. А Тамара жила за городом, потому что ее отцу, иноплеменнику,'),
 ('dataset/не дозволялось обитать среди иудеев, и ее мягкие, длинные локоны были нежного тем

In [ ]:
dataset_path = r"dataset/"

data = []

for j in [i.split(",") for i in texts.split("\n") if len(i.split(",")) == 2]:
    with open(dataset_path + j[1], "r") as t:
        text = t.read()
        text = text.replace("\n", "")
        data.append((dataset_path + j[0], text))

In [15]:
import random

In [18]:
random.shuffle(a2)

test_c = int(min(len(a2) * test_p, max_c))
val_c = int(min(len(a2) * val_p, max_c))

test = a2[:test_c]
val = a2[test_c: test_c + val_c]
train = a2[test_c + val_c:]

In [19]:
test_c

200

In [18]:
a = normalizer(text)

In [19]:
a

'По словам се оу и главного инженера старши Илона Маска, п в будущем заменит фалкон девять и фалкон хеви грузовой и пилотируемый сругин,'

In [25]:
_whitespace_re = re.compile(r'[^a-zA-Z0-9А-Яа-яЁё]+')

In [27]:
a = " ".join(re.split(_whitespace_re, text))
a

'По словам CEO и главного инженера SpaceX Илона Маска Starship в будущем заменит Falcon 9 и Falcon Heavy грузовой и пилотируемый Dragon V2 11 '

In [28]:
normalizer(a)

'По словам се оу и главного инженера спасекс Илона Маска пятшип в будущем заменит фалкон девять и фалкон хеви грузовой и пилотируемый драгон в дв грузовой и пилотируемый в одиннадцать '

In [30]:
a = normalizer(a)
a

'По словам се оу и главного инженера спасекс Илона Маска пятшип в будущем заменит фалкон девять и фалкон хеви грузовой и пилотируемый драгон в дв грузовой и пилотируемый в одиннадцать '

In [32]:
a.split(" ")

['По',
 'словам',
 'се',
 'оу',
 'и',
 'главного',
 'инженера',
 'спасекс',
 'Илона',
 'Маска',
 'пятшип',
 'в',
 'будущем',
 'заменит',
 'фалкон',
 'девять',
 'и',
 'фалкон',
 'хеви',
 'грузовой',
 'и',
 'пилотируемый',
 'драгон',
 'в',
 'дв',
 'грузовой',
 'и',
 'пилотируемый',
 'в',
 'одиннадцать',
 '']

'PA SLAVAM S0I AU I GLAVNAVA INZHYN0IRA SPAS0IKS ILANA MASKA P0ITSHYP F BUDUSH0IM ZAM0IN0IT FALKAN D0IV0IT0 I FALKAN KH0IV0I GRUZAVAJ0 I P0ILAT0IRUJ0IMYJ0 DRAGAN F TF GRUZAVAJ0 I P0ILAT0IRUJ0IMYJ0 F AD0INATSAT0'

In [2]:
norm = Normalizer()

In [12]:
text = norm.norm_text(text)
print(text)

через сто тысяч лет жизнь на земле оборвётся и семь ноль ноль ноль ноль ноль пятьсот людей умрёт


In [14]:
text

In [7]:

a(text)

['S0',
 'I',
 'V',
 'A',
 'D',
 'N0',
 'A',
 'D',
 'V',
 'A',
 'TS',
 'A',
 'T0',
 'P0',
 'I',
 'T',
 'A',
 'V',
 'A',
 'I',
 'J0',
 'U',
 'N0',
 'A',
 'M',
 'Y',
 'D0',
 'I',
 'L',
 'A',
 'J0',
 'I',
 'M',
 'P',
 'R',
 'A',
 'G',
 'R',
 'A',
 'M',
 'U',
 'D0',
 'I',
 'P',
 'R0',
 'I',
 'K',
 'A',
 'SH',
 'Y',
 'N',
 'V',
 'A',
 'R',
 'N0',
 'I',
 'N',
 'G',
 'D',
 'V',
 'A']